In [1]:
# !apt update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
# !tar -xvf spark-3.3.0-bin-hadoop3.tgz
# !pip install -q findspark
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"
import findspark
findspark.init()

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive', force_remount=True)

In [3]:
# %cd '/content/gdrive/My Drive/LDS9/Practice/Chapter6/'

In [4]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql import types
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import LinearRegressionModel

In [5]:
sc =SparkContext()

In [6]:
spark = SparkSession(sc)

## Chuẩn bị, chuẩn hóa dữ liệu, xác định input, output

In [7]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("flights.csv", inferSchema=True,header=True)

In [8]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- mon: integer (nullable = true)
 |-- dom: integer (nullable = true)
 |-- dow: integer (nullable = true)
 |-- carrier: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- org: string (nullable = true)
 |-- mile: integer (nullable = true)
 |-- depart: double (nullable = true)
 |-- duration: integer (nullable = true)
 |-- delay: string (nullable = true)



In [9]:
print((data.count(), len(data.columns)))

(50000, 10)


In [10]:
# Khi in bằng head thì định dạng hiển thị là row (khác với head ở pandas dataframe)
data.head(1)

[Row(mon=11, dom=20, dow=6, carrier='US', flight=19, org='JFK', mile=2153, depart=9.48, duration=351, delay='NA')]

In [11]:
data.show(3)

+---+---+---+-------+------+---+----+------+--------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|
+---+---+---+-------+------+---+----+------+--------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8|
+---+---+---+-------+------+---+----+------+--------+-----+
only showing top 3 rows



In [12]:
for item in data.head():
    print(item)

11
20
6
US
19
JFK
2153
9.48
351
NA


In [13]:
from pyspark.sql.functions import round

In [14]:
data = data.withColumn('km', round(data.mile*1.60934,0))

In [15]:
data = data.withColumn('label', (data.delay > 0).cast('integer'))

In [16]:
data.show(3)

+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|label|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|3465.0| NULL|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8| 542.0|    0|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+
only showing top 3 rows



In [17]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

In [18]:
#create Indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_inx')
indexer_model = indexer.fit(data)
data_indexed = indexer_model.transform(data)

In [19]:
#Repeat the process for orther categorical feature
data_indexed =  StringIndexer(inputCol='org', 
                              outputCol='org_inx').fit(data_indexed).transform(data_indexed)

In [20]:
# Create encoder
data_indexed = OneHotEncoder(inputCol='carrier_inx', 
                             outputCol='carrier_vec',
                            dropLast=True).fit(data_indexed).transform(data_indexed)

In [21]:
data_indexed = OneHotEncoder(inputCol='org_inx', 
                             outputCol='org_vec',
                            dropLast=True).fit(data_indexed).transform(data_indexed)

In [22]:
data_indexed.show(3)

+---+---+---+-------+------+---+----+------+--------+-----+------+-----+-----------+-------+-------------+-------------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|label|carrier_inx|org_inx|  carrier_vec|      org_vec|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+-----------+-------+-------------+-------------+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|3465.0| NULL|        6.0|    2.0|(8,[6],[1.0])|(7,[2],[1.0])|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|(8,[0],[1.0])|(7,[0],[1.0])|
|  2| 20|  4|     UA|   226|SFO| 337|  6.17|      82|   -8| 542.0|    0|        0.0|    1.0|(8,[0],[1.0])|(7,[1],[1.0])|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+-----------+-------+-------------+-------------+
only showing top 3 rows



In [23]:
data_indexed.columns

['mon',
 'dom',
 'dow',
 'carrier',
 'flight',
 'org',
 'mile',
 'depart',
 'duration',
 'delay',
 'km',
 'label',
 'carrier_inx',
 'org_inx',
 'carrier_vec',
 'org_vec']

In [24]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [25]:
assembler = VectorAssembler(
    inputCols=['mon', 'dom', 'dow','carrier_vec', 'org_vec','km','depart','duration'],
    outputCol='features'
)

In [26]:
data_pre = assembler.transform(data_indexed)

In [27]:
data_pre.select('features').show(2, truncate=False)

+--------------------------------------------------------------------+
|features                                                            |
+--------------------------------------------------------------------+
|(21,[0,1,2,9,13,18,19,20],[11.0,20.0,6.0,1.0,1.0,3465.0,9.48,351.0])|
|(21,[1,2,3,11,18,19,20],[22.0,2.0,1.0,1.0,509.0,16.33,82.0])        |
+--------------------------------------------------------------------+
only showing top 2 rows



In [28]:
data_pre.show(2)

+---+---+---+-------+------+---+----+------+--------+-----+------+-----+-----------+-------+-------------+-------------+--------------------+
|mon|dom|dow|carrier|flight|org|mile|depart|duration|delay|    km|label|carrier_inx|org_inx|  carrier_vec|      org_vec|            features|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+-----------+-------+-------------+-------------+--------------------+
| 11| 20|  6|     US|    19|JFK|2153|  9.48|     351|   NA|3465.0| NULL|        6.0|    2.0|(8,[6],[1.0])|(7,[2],[1.0])|(21,[0,1,2,9,13,1...|
|  0| 22|  2|     UA|  1107|ORD| 316| 16.33|      82|   30| 509.0|    1|        0.0|    0.0|(8,[0],[1.0])|(7,[0],[1.0])|(21,[1,2,3,11,18,...|
+---+---+---+-------+------+---+----+------+--------+-----+------+-----+-----------+-------+-------------+-------------+--------------------+
only showing top 2 rows



In [29]:
final_data = data_pre.select('features','label')

In [30]:
final_data.count()

50000

In [31]:
final_data = final_data.na.drop()
final_data.count()

47022

In [32]:
new_data = data_pre.select('features','label').filter(data_pre.label.isNull())
new_data.show(3,False)

+--------------------------------------------------------------------+-----+
|features                                                            |label|
+--------------------------------------------------------------------+-----+
|(21,[0,1,2,9,13,18,19,20],[11.0,20.0,6.0,1.0,1.0,3465.0,9.48,351.0])|NULL |
|(21,[0,1,2,4,11,18,19,20],[4.0,2.0,5.0,1.0,1.0,415.0,8.92,65.0])    |NULL |
|(21,[1,2,3,11,18,19,20],[8.0,2.0,1.0,1.0,538.0,11.08,85.0])         |NULL |
+--------------------------------------------------------------------+-----+
only showing top 3 rows



In [33]:
new_data.count()

2978

In [34]:
train_data, test_data = final_data.randomSplit([0.8,0.2])

In [35]:
train_data.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|             37606|
|   mean|0.6377971600276552|
| stddev|0.4806434081583298|
|    min|                 0|
|    max|                 1|
+-------+------------------+



In [36]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|               9416|
|   mean| 0.6363636363636364|
| stddev|0.48107123901325205|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [37]:
from pyspark.ml.classification import LogisticRegression

In [38]:
logistic = LogisticRegression(featuresCol='features',
                         labelCol='label',
                         predictionCol='prediction')

In [39]:
logisticModel = logistic.fit(train_data)

In [58]:
test_model = logisticModel.transform(test_data)
test_model.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|  556|
|    0|       0.0|  697|
|    1|       1.0| 5383|
|    0|       1.0| 2704|
+-----+----------+-----+



In [59]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,  BinaryClassificationEvaluator

In [60]:
multi_evaluator = MulticlassClassificationEvaluator()
acc_ = multi_evaluator.evaluate(test_model,
                               {multi_evaluator.metricName: 'accuracy'})

In [61]:
acc_

0.6509635974304069

In [62]:
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(test_model,
                               {multi_evaluator.metricName: 'areaUnderROC'})

In [63]:
auc

0.642383441693481

In [64]:
logisticModel.save('logisticModel_Flights_50k')

In [65]:
from pyspark.ml.classification import LogisticRegressionModel
logisticModel2 = LogisticRegressionModel.load('logisticModel_Flights_50k')

In [66]:
unlabeled_data = new_data.select('features')

In [67]:
predictions = logisticModel2.transform(unlabeled_data)

In [69]:
predictions[['features','probability','prediction']].show(5,False)

+--------------------------------------------------------------------+----------------------------------------+----------+
|features                                                            |probability                             |prediction|
+--------------------------------------------------------------------+----------------------------------------+----------+
|(21,[0,1,2,9,13,18,19,20],[11.0,20.0,6.0,1.0,1.0,3465.0,9.48,351.0])|[0.606156751478692,0.39384324852130803] |0.0       |
|(21,[0,1,2,4,11,18,19,20],[4.0,2.0,5.0,1.0,1.0,415.0,8.92,65.0])    |[0.35470708890989233,0.6452929110901077]|1.0       |
|(21,[1,2,3,11,18,19,20],[8.0,2.0,1.0,1.0,538.0,11.08,85.0])         |[0.3143064747227623,0.6856935252772377] |1.0       |
|(21,[0,1,3,11,18,19,20],[5.0,8.0,1.0,1.0,378.0,14.48,79.0])         |[0.3429578499304015,0.6570421500695984] |1.0       |
|(21,[0,1,2,9,14,18,19,20],[1.0,13.0,3.0,1.0,1.0,344.0,20.0,82.0])   |[0.30768130574146285,0.6923186942585371]|1.0       |
+---------------